In [19]:
import pandas as pd
import numpy as np

In [21]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.impute import SimpleImputer 

In [3]:
df=pd.read_csv('/content/covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df['gender'].value_counts()

Female    59
Male      41
Name: gender, dtype: int64

In [6]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [7]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

Preprocessing of data:

City is nominal data we use OneHot Encoder()

cough is ordinal data we use OrdinalEncoder ()

Fever have some missing column for this we use SimpleImputer(

In [15]:
# Before applying any transformer we first split our data in train and test

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split (df.drop('has_covid', axis = 1),df['has_covid'], test_size=0.2,random_state=True)

In [16]:
X_train.shape

(80, 5)

In [17]:
X_test.shape

(20, 5)

#Performing seprate opertaion on each columns
##1. Simple Imputer for filling Nan value

In [18]:
X_train.isnull().sum()

age       0
gender    0
fever     8
cough     0
city      0
dtype: int64

In [25]:
si= SimpleImputer()

X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

##2. Ordinal Encoding for ordinal columns i.e cough

In [45]:
# Ordinal encoding -> Cough

oe=OrdinalEncoder(categories=[['Mild','Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])
# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

##3. OneHot Encoding

In [34]:
#OHE -> 
ohe = OneHotEncoder(drop='first',sparse=False)

X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [41]:
X_train_age=X_train.drop(columns=['gender','city','cough','fever']).values
X_test_age=X_test.drop(columns=['gender','city','cough','fever']).values

In [48]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_cough,X_train_gender_city), axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough), axis=1)

In [58]:
X_train_transformed.shape

(80, 7)

#Performing the above operation using Column Transformer

In [49]:
from sklearn.compose import ColumnTransformer

In [54]:
transformer  = ColumnTransformer (transformers=[
     ('tnf1',SimpleImputer(),['fever']),
     ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
     ('tnf3',OneHotEncoder(drop='first',sparse=False),['city','gender'])                                           
], remainder='passthrough')

In [56]:
transformer.fit_transform(X_train).shape

(80, 7)

In [57]:
transformer.fit_transform(X_test).shape

(20, 7)